In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [88]:
df = pd.read_csv('/home/iamtheuserofthis/python_workspace/data_for_ml/titanic/train.csv')

In [133]:
df=df[df['Embarked'].notna()]
y = df['Survived']
X = df[['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Cabin', 'Embarked']]

X

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,NaN,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,NaN,1,2,23.4500,NaN,S
889,1,male,26.0,0,0,30.0000,C148,C


In [90]:
X['Cabin'][0] is np.nan

True

In [91]:
X['Cabin'].map(lambda x: (x is not np.nan)).values.astype(int)

array([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,

In [186]:
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
class ExperimentalTransformer(BaseEstimator,TransformerMixin):
    
    def fit(self,X,y=None):
        return self
    
    def transform(self,X,y=None):
        X_ = X.copy()
        X_['Cabin'] = X_['Cabin'].map(lambda x: (x is not np.nan)).values.astype(int)
        return X_
    

In [187]:
transformers_col = ColumnTransformer([
    ('one_hot_enc', OneHotEncoder(), ['Pclass', 'Embarked','Sex']),
    ('simple_imputer',SimpleImputer(strategy='median'),['Age']),
    ('cabin',  ExperimentalTransformer(), ['Cabin']),
])
df_1 = transformers_col.fit_transform(X)

In [188]:
# pd.DataFrame(df_1, columns = ['Pclass1', 'Pclass2','Pclass3','Embarked1','Embarked2','Embarked3','SexM','SexF','Age'])
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2)

In [189]:
from sklearn.ensemble import RandomForestClassifier
X_train_processed = transformers_col.fit_transform(X_train)
rfc = RandomForestClassifier()


In [268]:
transformers_col.named_transformers_['one_hot_enc']\
.get_feature_names(['Pclass', 'Embarked','Sex'])\
.tolist() + ['Age'] +['Cabin']

def get_feat_names(col_transformers,transformer):
    col_list = []
    print(col_transformers)
    for tx in col_transformers:
        if type(transformer.named_transformers_[tx]) is OneHotEncoder:
            col_list.append(transformer.named_transformers_[tx].get_feature_names(col_transformers[tx]).tolist())
        else:
            col_list = col_list +  col_transformers[tx]
    return col_list

In [269]:
transformer_names = {'one_hot_enc':['Pclass','Embarked','Sex'], 'simple_imputer':['Age'], 'cabin':['Cabin']}


#pd.DataFrame(X_train_processed)
get_feat_names(transformer_names, transformers_col)

{'one_hot_enc': ['Pclass', 'Embarked', 'Sex'], 'simple_imputer': ['Age'], 'cabin': ['Cabin']}


[['Pclass_1',
  'Pclass_2',
  'Pclass_3',
  'Embarked_C',
  'Embarked_Q',
  'Embarked_S',
  'Sex_female',
  'Sex_male'],
 'Age',
 'Cabin']